<a href="https://colab.research.google.com/github/gizemkaratas/DATA-LEARNING/blob/main/encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#############################################
# 3. Encoding (Label Encoding, One-Hot Encoding, Rare Encoding)
#############################################

#############################################
# Label Encoding & Binary Encoding
#############################################

# değişken değeri  0-1 den ibaretse bunu binary encodingle yaparız label encod 2 ve üzeri için kullanılır ikisi aynı şeydir ama label encod daha kapsamlıdır.Kategorik değişkenlerin sınıfı labeldır.
#ordinal sıralı kategorik değişkeni label encoder ile dönüştürürüz. örn eğitim seviyesi gibi.nominal değişkenler  label encoderdan geçmemelidir yoksa sonuçlar yanlış çıkacaktı örneğin gs ye fazla değer atayıp sanki daha önemli bir takım değişkenmiş gibi önceliklendirecektir.
df = load()
df.head()
df["Sex"].head()

le = LabelEncoder() #alfabetik olarak 1,2,3 olarak numaralandırır
le.fit_transform(df["Sex"])[0:5]
le.inverse_transform([0, 1]) #hangisine 1 hangisine 2 vs dediğini unutursak bu şekilde sorabiliriz örn female 0 demiş f daha önce olduğu için m den

def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

df = load()

binary_cols = [col for col in df.columns if df[col].dtype not in [int, float] # eğer değişkkende 2 sınıf varsa bunu bul ve binary encoder uygula, eğer içinde nan değer varsa buna da ayrı değer atayabilir bu sebeple ya öncesinde nanlara işlem yapmak ya da bunun farkında olmak gerekir.
               and df[col].nunique() == 2]

for col in binary_cols:
    label_encoder(df, col)

df.head()

df = load_application_train()
df.shape

binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

df[binary_cols].head()


for col in binary_cols:
    label_encoder(df, col)


df = load()
df["Embarked"].value_counts()
df["Embarked"].nunique()
len(df["Embarked"].unique())

In [ ]:
#############################################
# One-Hot Encoding
#############################################

df = load()
df.head()
df["Embarked"].value_counts()

pd.get_dummies(df, columns=["Embarked"]).head() # embarked değişkenini getir  ona one encod yap ve tekrar dfye otomatik tanımlar

pd.get_dummies(df, columns=["Embarked"], drop_first=True).head() #kukla değişken olan ilk sütunu uçurur c,q,s sınıfı var embarkedda c alfabetik olarak önde ve kukla değişken onu sildi dfden

pd.get_dummies(df, columns=["Embarked"], dummy_na=True).head() #nan değerin de bir sınıf olarak alınmasını istiyorsak dumy_na=true deriz

pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True).head() # bu şekilde 2 sınıflı sex değişkeni için binary encod (one hot yapınca ilk sütun uçuyor bu yüzden otomatik binary oluyor 2 sınıflılar), embarked için deotomatik one hot yaptık tek sütunda

def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

df = load()

# cat_cols, num_cols, cat_but_car = grab_col_names(df)

ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2] #2 den fazla 10 dan az sınıflıları getirip encod et


one_hot_encoder(df, ohe_cols).head()

df.head() #eğer bu işlemin dfte kalıcı kalmasını istiyorsak df=one_hot yapmalıyız

In [ ]:
#############################################
# Rare Encoding
#############################################

# 1. Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.
# 2. Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
# 3. Rare encoder yazacağız.

###################
# 1. Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.
###################

df = load_application_train()
df["NAME_EDUCATION_TYPE"].value_counts()

cat_cols, num_cols, cat_but_car = grab_col_names(df) #kategorik değişkenleri bul

def cat_summary(dataframe, col_name, plot=False): # sınıf frekab oranı düşüşk olanlaru birleştirip rare yapabiliriz sınıfı çıkarabiliriz bilgi taşımayan değişkenleri silebiliriz. (bu tüm veriye sınıfın etki oranı gözlenme oranını verir)
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()


for col in cat_cols:
    cat_summary(df, col)

###################
# 2. Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
###################

df["NAME_INCOME_TYPE"].value_counts()

df.groupby("NAME_INCOME_TYPE")["TARGET"].mean() #bağımlı değişkene göre etki değerlendirmesi yapıyoruz , birbirine yakın oranı veren  sınıfların bşrleştirilmesi dwha mantıklıdır ama bazen tamamnını da birleştiririz


def rare_analyser(dataframe, target, cat_cols): #rare analizi 1. işlemdekileri fonksiyonlaştırdı
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

rare_analyser(df, "TARGET", cat_cols)

#############################################
# 3. Rare encoder'ın yazılması.
#############################################

def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

new_df = rare_encoder(df, 0.01) # oran değeri 0,01 altındwa olan sınıflaru birleştir

rare_analyser(new_df, "TARGET", cat_cols)

df["OCCUPATION_TYPE"].value_counts()
